In [ ]:

import olorenchemengine as oce
from tqdm import tqdm
from tdc.benchmark_group import admet_group
import numpy as np
import pandas as pd



In [ ]:
models = {
    'BBB_TDC': '{"BC_class_name": "BaseBoosting", "args": [[{"BC_class_name": "RandomForestModel", "args": [{"BC_class_name": "DescriptastorusDescriptor", "args": ["rdkit2dnormalized"], "kwargs": {}}], "kwargs": {"n_estimators": 1000, "max_features": "log2", "max_depth": null, "criterion": "entropy", "class_weight": null, "bootstrap": true}}, {"BC_class_name": "RandomForestModel", "args": [{"BC_class_name": "DescriptastorusDescriptor", "args": ["morgan3counts"], "kwargs": {}}], "kwargs": {"n_estimators": 1000, "max_features": "log2", "max_depth": null, "criterion": "entropy", "class_weight": null, "bootstrap": true}}, {"BC_class_name": "RandomForestModel", "args": [{"BC_class_name": "OlorenCheckpoint", "args": ["default"], "kwargs": {"num_tasks": 2048}}], "kwargs": {"n_estimators": 1000, "max_features": "log2", "max_depth": null, "criterion": "entropy", "class_weight": null, "bootstrap": true}}]], "kwargs": {"n": 1, "log": true}}',
    'BBB_1040':"{'BC_class_name': 'BaseBoosting', 'args': [[{'BC_class_name': 'RandomForestModel', 'args': [{'BC_class_name': 'DescriptastorusDescriptor', 'args': ['rdkit2dnormalized'], 'kwargs': {'log': True, 'scale': None}}], 'kwargs': {'n_estimators': 1000, 'max_features': 'log2', 'max_depth': None, 'criterion': 'entropy', 'class_weight': None, 'bootstrap': True}}, {'BC_class_name': 'RandomForestModel', 'args': [{'BC_class_name': 'DescriptastorusDescriptor', 'args': ['morgan3counts'], 'kwargs': {'log': True, 'scale': None}}], 'kwargs': {'n_estimators': 1000, 'max_features': 'log2', 'max_depth': None, 'criterion': 'entropy', 'class_weight': None, 'bootstrap': True}}, {'BC_class_name': 'RandomForestModel', 'args': [{'BC_class_name': 'OlorenCheckpoint', 'args': ['default'], 'kwargs': {'num_tasks': 2048, 'log': True}}], 'kwargs': {'n_estimators': 1000, 'max_features': 'log2', 'max_depth': None, 'criterion': 'entropy', 'class_weight': None, 'bootstrap': True}}]], 'kwargs': {'n': 1, 'log': True, 'oof': False, 'nfolds': 5}}",
    'BBB_455': "{'BC_class_name': 'RandomForestModel', 'args': [{'BC_class_name': 'DescriptastorusDescriptor', 'args': ['rdkit2dnormalized'], 'kwargs': {'log': True, 'scale': None}}], 'kwargs': {'n_estimators': 2000, 'max_features': 'log2', 'max_depth': None, 'criterion': 'entropy', 'class_weight': None, 'bootstrap': True}}",
    'BBB_1209':"{'BC_class_name': 'ChemPropModel', 'args': [], 'kwargs': {'epochs': 100, 'dropout_rate': 0.0, 'batch_size': 50, 'lr': 0.001, 'hidden_size': 300, 'depth': 3}}"
}

In [ ]:
model_dict = {
    'bbb_base': models["BBB_TDC"],
    'bbb_1040': models["BBB_1040"],
    'bbb_455' : models["BBB_455"],
    'bbb_1209': models["BBB_1209"],
}

In [ ]:
def model_from_dict(s):
    s = s.replace("'", "\"").replace("True", "true").replace("False", "false").replace("None", "null")
    model = oce.create_BC(s)
    return model

In [ ]:
#EDIT THIS CELL
TASK_NAME = 'bbb_martins'

In [ ]:

results_dict = {}
for MODEL in model_dict.keys():
    print(f"__________________________")
    group = admet_group(path = 'data/')
    predictions_list = []
    for seed in tqdm([1, 2, 3, 4, 5]):
        benchmark = group.get(TASK_NAME) 
        predictions = {}
        name = benchmark['name']
        train, test = benchmark['train_val'], benchmark['test']
        model = model_from_dict(model_dict[MODEL])
        model.fit(train['Drug'], train['Y'])
        y_pred_test = model.predict(test['Drug'])
        oce.save(model, MODEL)
        predictions[name] = y_pred_test
        predictions_list.append(predictions)
    results = group.evaluate_many(predictions_list)
    results_dict[MODEL] = results[TASK_NAME]
print(results_dict)

Baseline Benchmark Results
-------------
|BBB Baseline Model|Error|
|-|-|
|bbb_base |[0.922, 0.001]|
|bbb_1040 |[0.922, 0.001]|
|bbb_455  |[0.911, 0.000]|
|bbb_1209 |[0.870, 0.003]|


